In [27]:
import tifffile
import zarr

In [15]:
z = zarr.open('/mnt/raid_nvme/inference_out/predictions.zarr', mode='r')
print(z.tree())

norms = z['normals_final']
sheet = z['sheet_final']

s_n = norms[:,1000, :, ]
s_s = sheet[1000]

tifffile.imwrite('s_n.tif', s_n)
tifffile.imwrite('s_s.tif', s_s)

/
 ├── normals_count (2000, 1500, 1750) float32
 ├── normals_final (3, 2000, 1500, 1750) uint16
 ├── normals_sum (3, 2000, 1500, 1750) float32
 ├── sheet_count (2000, 1500, 1750) float32
 ├── sheet_final (2000, 1500, 1750) uint8
 └── sheet_sum (2000, 1500, 1750) float32


In [28]:
#raw = zarr.open('/home/sean/Documents/GitHub/VC-Surface-Models/models/normals/s4_sheet.zarr/volume.zarr', mode='r')
raw = zarr.open("/home/sean/Desktop/s1_segments/1619/.zarr", mode='r')
t = raw['layers'][10]
tifffile.imwrite('s4_sheet_10.tif', t)


AttributeError: 

In [4]:
import shutil
import os

def copy_and_number(source_image, start_number, num_copies):
    # Check if source file exists
    if not os.path.exists(source_image):
        print(f"Error: Source file '{source_image}' not found")
        return

    # Get the directory and file extension
    directory = os.path.dirname(source_image) or '.'
    _, ext = os.path.splitext(source_image)

    # Create numbered copies
    for i in range(num_copies):
        new_number = start_number + i
        new_filename = f"{new_number}{ext}"
        new_path = os.path.join(directory, new_filename)

        try:
            shutil.copy2(source_image, new_path)
            print(f"Created: {new_filename}")
        except Exception as e:
            print(f"Error creating {new_filename}: {e}")

# Example usage
if __name__ == "__main__":
    source_image = "/home/sean/Desktop/s1_segments/5753/20230702185753_inklabels.png"
    start_number = 1
    num_copies =20

    copy_and_number(source_image, start_number, num_copies)

Created: 1.png
Created: 2.png
Created: 3.png
Created: 4.png
Created: 5.png
Created: 6.png
Created: 7.png
Created: 8.png
Created: 9.png
Created: 10.png
Created: 11.png
Created: 12.png
Created: 13.png
Created: 14.png
Created: 15.png
Created: 16.png
Created: 17.png
Created: 18.png
Created: 19.png
Created: 20.png


In [13]:

import zarr
import numpy as np
def check_slices_zyx3_vs_zyx(normals_path, sheet_path):
    """
    Compares two Zarr arrays:
      - 'normals': shape (Z, Y, X, 3)
      - 'sheet':   shape (Z, Y, X)

    For each z in [0..Z-1], checks if there's nonzero data in that slice.
    Prints which slices are nonempty in one array but empty in the other.
    """

    normals_arr = zarr.open(normals_path, mode='r')
    sheet_arr = zarr.open(sheet_path, mode='r')

    # Basic shape checks
    if normals_arr.ndim != 4 or sheet_arr.ndim != 3:
        raise ValueError(
            f"Shapes must be (Z, Y, X, 3) and (Z, Y, X). "
            f"Got normals={normals_arr.shape}, sheet={sheet_arr.shape}"
        )
    if normals_arr.shape[0] != sheet_arr.shape[0] or \
       normals_arr.shape[1] != sheet_arr.shape[1] or \
       normals_arr.shape[2] != sheet_arr.shape[2]:
        raise ValueError(
            f"Z, Y, X dimensions do not match.\n"
            f"normals (Z,Y,X,3): {normals_arr.shape}\n"
            f"sheet   (Z,Y,X):   {sheet_arr.shape}"
        )

    depth = normals_arr.shape[0]  # Number of slices in Z

    for z in range(depth):
        # Read one slice from each array
        # normals_slice: shape (Y, X, 3)
        # sheet_slice:   shape (Y, X)
        normals_slice = normals_arr[z, ...]  # (Y, X, 3)
        sheet_slice = sheet_arr[z, ...]      # (Y, X)

        # Check if there's any nonzero data
        has_data_normals = np.any(normals_slice)
        has_data_sheet = np.any(sheet_slice)

        if has_data_normals and not has_data_sheet:
            print(f"Slice {z}: has data in normals, but not in sheet.")
        elif has_data_sheet and not has_data_normals:
            print(f"Slice {z}: has data in sheet, but not in normals.")
        # If both or neither have data, we do nothing

    print("Done checking slices!")

check_slices_zyx3_vs_zyx('/home/sean/Documents/GitHub/VC-Surface-Models/models/normals/s4_normals.zarr/volume.zarr', '/home/sean/Documents/GitHub/VC-Surface-Models/models/normals/s4_sheet.zarr/volume.zarr')

KeyboardInterrupt: 